<a href="https://colab.research.google.com/github/Harshkotkar/Deep-Learning/blob/main/Early_Detection_of_Diabetic_Retinopathy_from_Retinal_Fundus_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Early Detection of Diabetic Retinopathy from Retinal Fundus Images

In [4]:
# Run this cell first to set up Google Colab
!pip install -q kaggle albumentations opencv-python-headless
!apt install -qq unzip

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import zipfile
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
TensorFlow version: 2.19.0
Num GPUs Available:  1


In [7]:
# Upload your kaggle.json file
from google.colab import files
files.upload()  # This will prompt you to upload your kaggle.json file

# Move kaggle.json to the proper location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle_key.json

# Download the dataset
!kaggle competitions download -c aptos2019-blindness-detection

# Create directories
!mkdir -p train_images test_images

# Extract the files
with zipfile.ZipFile('aptos2019-blindness-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# Extract train images
with zipfile.ZipFile('train_images.zip', 'r') as zip_ref:
    zip_ref.extractall('train_images')

# Extract test images
with zipfile.ZipFile('test_images.zip', 'r') as zip_ref:
    zip_ref.extractall('test_images')

print("Dataset downloaded and extracted successfully!")

Saving kaggle_key.json to kaggle_key.json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle_key.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


FileNotFoundError: [Errno 2] No such file or directory: 'aptos2019-blindness-detection.zip'

In [9]:
# First, let's check what files we have
import os
print("Files in current directory:", os.listdir('.'))

# Fix the kaggle.json issue
if 'kaggle_key.json' in os.listdir('.'):
    !mkdir -p ~/.kaggle
    !cp kaggle_key.json ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    print("Fixed kaggle.json configuration")
else:
    print("kaggle_key.json not found. Please upload it again.")

Files in current directory: ['.config', 'train_images', 'test_images', 'kaggle_key.json', '.ipynb_checkpoints', 'sample_data']
Fixed kaggle.json configuration


In [3]:
# # Define the dataset class
# class DRDataset:
#     def __init__(self, csv_file, img_dir, img_size=(256, 256)):
#         self.data = pd.read_csv(csv_file)
#         self.img_dir = img_dir
#         self.img_size = img_size

#         # Calculate class weights for handling imbalance
#         class_counts = self.data['diagnosis'].value_counts().sort_index()
#         total_samples = len(self.data)
#         self.class_weights = {i: total_samples / (5 * count) for i, count in enumerate(class_counts)}

#     def __len__(self):
#         return len(self.data)

#     def load_image(self, img_name):
#         img_path = os.path.join(self.img_dir, img_name + '.png')
#         image = Image.open(img_path).convert('RGB')
#         return image

#     def preprocess_image(self, image):
#         # Crop black borders
#         image = np.array(image)
#         gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#         _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
#         contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         if contours:
#             cnt = contours[0]
#             x, y, w, h = cv2.boundingRect(cnt)
#             image = image[y:y+h, x:x+w]

#         # Resize and normalize
#         image = Image.fromarray(image)
#         image = image.resize(self.img_size)
#         image = np.array(image) / 255.0
#         return image

#     def get_class_weights(self):
#         return self.class_weights

# # Create data generators
# def create_data_generators(csv_path, img_dir, batch_size=32, validation_split=0.2, img_size=(256, 256)):
#     # Load dataset
#     dataset = DRDataset(csv_path, img_dir, img_size)

#     # Split data
#     train_df, val_df = train_test_split(
#         dataset.data,
#         test_size=validation_split,
#         stratify=dataset.data['diagnosis'],
#         random_state=42
#     )

#     # Get class weights
#     class_weights = dataset.get_class_weights()

#     # Create data generators
#     train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#         preprocessing_function=None,
#         rotation_range=20,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         horizontal_flip=True,
#         vertical_flip=True,
#         brightness_range=[0.8, 1.2],
#         zoom_range=0.2,
#         fill_mode='constant',
#         cval=0
#     )

#     val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#     train_generator = train_datagen.flow_from_dataframe(
#         dataframe=train_df,
#         directory=img_dir,
#         x_col='id_code',
#         y_col='diagnosis',
#         target_size=img_size,
#         batch_size=batch_size,
#         class_mode='raw',
#         shuffle=True,
#         seed=42
#     )

#     val_generator = val_datagen.flow_from_dataframe(
#         dataframe=val_df,
#         directory=img_dir,
#         x_col='id_code',
#         y_col='diagnosis',
#         target_size=img_size,
#         batch_size=batch_size,
#         class_mode='raw',
#         shuffle=False
#     )

#     return train_generator, val_generator, class_weights

# # Load dataset and analyze class distribution
# csv_path = 'train.csv'  # Update with your path
# img_dir = 'train_images'  # Update with your path

# dataset = DRDataset(csv_path, img_dir)
# print(f"Dataset size: {len(dataset)}")

# # Analyze class distribution
# class_counts = dataset.data['diagnosis'].value_counts().sort_index()
# print("Class distribution:")
# for i, count in enumerate(class_counts):
#     print(f"Class {i}: {count} samples ({count/len(dataset)*100:.2f}%)")

# plt.figure(figsize=(10, 6))
# sns.barplot(x=class_counts.index, y=class_counts.values)
# plt.title('Class Distribution in APTOS 2019 Dataset')
# plt.xlabel('DR Severity Level')
# plt.ylabel('Number of Images')
# plt.show()

In [12]:
# Now let's download the dataset using the Kaggle API
!kaggle competitions download -c aptos2019-blindness-detection

# Check if download was successful
import os
print("Files after download:", os.listdir('.'))

# Extract the files
import zipfile

# Extract the main zip file
with zipfile.ZipFile('aptos2019-blindness-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

print("Files after extraction:", os.listdir('.'))

# Extract train images
if 'train_images.zip' in os.listdir('.'):
    with zipfile.ZipFile('train_images.zip', 'r') as zip_ref:
        zip_ref.extractall('train_images')
    print("Train images extracted successfully!")
else:
    print("train_images.zip not found")

# Extract test images
if 'test_images.zip' in os.listdir('.'):
    with zipfile.ZipFile('test_images.zip', 'r') as zip_ref:
        zip_ref.extractall('test_images')
    print("Test images extracted successfully!")
else:
    print("test_images.zip not found")

# Check if we have the CSV file
if 'train.csv' in os.listdir('.'):
    print("CSV file found!")
else:
    print("train.csv not found")

# Let's check the contents of our directories
print("\nDirectory structure:")
print("Current directory:", os.listdir('.'))
if os.path.exists('train_images'):
    train_files = os.listdir('train_images')
    print(f"Train images directory: {len(train_files)} files")
    if train_files:
        print("Sample train files:", train_files[:5])
if os.path.exists('test_images'):
    test_files = os.listdir('test_images')
    print(f"Test images directory: {len(test_files)} files")
    if test_files:
        print("Sample test files:", test_files[:5])

^C
Files after download: ['.config', 'train_images', 'test_images', 'kaggle_key.json', '.ipynb_checkpoints', 'sample_data']


FileNotFoundError: [Errno 2] No such file or directory: 'aptos2019-blindness-detection.zip'

In [14]:
# Let's try a direct download approach (if available)
# Note: This might not work as Kaggle usually requires authentication

try:
    # Try to download using the Kaggle API again with the correct file name
    !mkdir -p ~/.kaggle
    !cp kaggle_key.json ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json

    # Check if authentication works
    !kaggle competitions list -s aptos2019
    print("Kaggle API setup successful!")

    # Now try to download
    !kaggle competitions download -c aptos2019-blindness-detection

    # Extract the files
    with zipfile.ZipFile('aptos2019-blindness-detection.zip', 'r') as zip_ref:
        zip_ref.extractall('.')

    # Extract train images
    with zipfile.ZipFile('train_images.zip', 'r') as zip_ref:
        zip_ref.extractall('train_images')

    # Extract test images
    with zipfile.ZipFile('test_images.zip', 'r') as zip_ref:
        zip_ref.extractall('test_images')

    print("Dataset downloaded and extracted successfully!")

except Exception as e:
    print(f"Error with Kaggle API: {e}")
    print("Let's try a different approach...")

    # Alternative: Use a pre-loaded version if available
    # Note: This is a long shot, but sometimes datasets are available via other means
    print("Please manually download the dataset from Kaggle and upload it")

No competitions found
Kaggle API setup successful!
^C
Error with Kaggle API: [Errno 2] No such file or directory: 'aptos2019-blindness-detection.zip'
Let's try a different approach...
Please manually download the dataset from Kaggle and upload it
